# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>
A two sample $z$-test is most appropriate for this problem.  The hypthesis test will determine whether there is a significant difference in the true proporition of callbacks between white sounding names and black sounding names.  
<br>
We assume that the Central Limit Theorem (CLT) does apply, based on the following conditions:

<ol>
  <li> <b>Independence</b> - We assume the callbacks are independent, and that the decision about whether to call back for any one resume does not affect the decision to call back for any other resume.</li>
  <li> <b>Randomization</b> - We assume the data is randomly sampled.  The names are randomly assigned to the resumes.  The experimental summary shows that the experimental design controls for many factors, such as quality of resume and type of employer. </li>
  <li> <b>Sample Size</b> - There are 4870 data points total, with 2435 data points each for black and white sounding names.  The sample size is large enough that the expected numbers of failures and successes are sufficiently large $(\ge 10)$ for each group, assuming the null hypothesis below. </li>
</ol>

A one-tailed test is performed, in order to test whether the proportion of callbacks for white sounding names is significantly higher than the proportion of callbacks for black sounding names.  The significance value of $\alpha=0.01$ is chosen in order to reduce the likelihood of a Type I error.  
<br>
The null and alternative hypotheses are:

<img src="http://latex.codecogs.com/svg.latex?H_0: p_w = p_b" border="0"/>
<img src="http://latex.codecogs.com/svg.latex?H_1: p_w > p_b" border="0"/>

<br>
where $p_w$ and $p_b$ are the true population callback proportions for white sounding names and black sounding names, respectively.
</p>
</div>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

In [6]:
# estimate the population proportions for the null hypothesis
p_0 = np.sum(data.call) / len(data.call)

# verify that each group has the same number of data points
n_w = len(w.call)
n_b = len(b.call)
print('n_w =', n_w,'   n_b =', n_b)

# print the expected number of successes for each group
nexp_b = p_0 * n_b
nexp_w = p_0 * n_w
print('Expected number of successes and failures for w =',
      (nexp_w, n_w - nexp_w))
print('Expected number of successes and failures for b =',
      (nexp_b, n_b - nexp_b))

# find the difference between sample proportions
p_w = np.mean(w.call)
p_b = np.mean(b.call)
p_wmb = p_w - p_b

# null hypothesis variance of the sampling distribution
var_p_wmb = p_0 * (1 - p_0) * ((1 / n_w) + (1 / n_b))

print('Observed proportion difference p_w - p_b =', p_wmb)

n_w = 2435    n_b = 2435
Expected number of successes and failures for w = (195.99999999999997, 2239.0)
Expected number of successes and failures for b = (195.99999999999997, 2239.0)
Observed proportion difference p_w - p_b = 0.0320329


### Bootstrap

In [7]:
print('\nBootstrap Summary \n')

# seed the pseudorandom number generator
np.random.seed(42)

# set the number of replicates to generate
NREPS = 10000

# initialize an array for the bootstrap replicates
bs_reps = np.empty(NREPS)

# generate 10000 bootstrap replicates for p_w - p_b
bs_reps_w = np.random.binomial(n_w, p_0, size=NREPS) / n_w
bs_reps_b = np.random.binomial(n_b, p_0, size=NREPS) / n_b
bs_reps = bs_reps_w - bs_reps_b
    
# evaluate the confidence interval
ci_bs = np.percentile(bs_reps + p_wmb, [0.5, 99.5])
print('99% Confidence Interval is', tuple(ci_bs))

# evaluate the margin of error
me_bs = (ci_bs[1] - ci_bs[0]) / 2
print('Margin of Error =', me_bs)

# evaluate the p-value
p_bs = np.sum(bs_reps >= p_wmb) / len(bs_reps)
print('p-value =', p_bs)


Bootstrap Summary 

99% Confidence Interval is (0.011909651569639634, 0.052566735758551336)
Margin of Error = 0.0203285420945
p-value = 0.0


### z-test

In [8]:
# perform a z-test
print('z-test Summary \n')

# evaluate the z-score 
z = (p_wmb - 0) / np.sqrt(var_p_wmb)
print('Observed z-score =', z)

# evaluate the 99% confidence interval
ci_z = stats.norm.interval(0.99, loc=p_wmb, scale=np.sqrt(var_p_wmb))
print('99% Confidence Interval is', ci_z)

# evaluate the margin of error
me_z = (ci_z[1] - ci_z[0]) / 2
print('Margin of Error =', me_z)

# evaluate the p-value
p_z = 1 - stats.norm.cdf(z)
print('p-value =', p_z)

z-test Summary 

Observed z-score = 4.10841223524
99% Confidence Interval is (0.011949386720027055, 0.052116322990094097)
Margin of Error = 0.020083468135
p-value = 1.99194270475e-05


<div class="span5 alert alert-success">
<p> 
A 99% confidence interval was evaluated for each of two different approaches. The corresponding margin of error was evaluated for each confidence interval as well.  Both the bootstrap approach and the frequentist approach had similar results, with a confidence interval of $(0.01, 0.05)$ and a margin of error of $0.02$.  These results suggest that, with 99% confidence, the true difference between the population proportions of callbacks for white sounding names and black sounding names is within this interval.  The interval does not include 0, so the confidence intervals suggest that the two population proportions are not equal.  
<br>
A $p$-value was obtained for each of two hypothesis tests.  The bootstrap hypothesis test gave a $p$-value of $0$, indicating that none of the 10,000 generated bootstrap replicates were at least as extreme as the difference between the observed sample proportions.  The $z$-test gave a $p$-value of $2.0 \times 10^{-5}$, indicating that there is a less than 1 % chance of observing a value as extreme as the difference between the sample proportions or greater under the null hypothesis.  <b>For each test, the null hypothesis is rejected and the alternative hypothesis is accepted.  The tests indicate that there is a significant difference between the population proportions of callbacks for white sounding names and black sounding names.  Specifically, these one-tailed tests indicate that the black sounding names have a lower population proportion of callbacks than the white sounding names.</b>  
<br>
This analysis does not provide an indication of the importance of race/name in callback success.  The $p$-values do not indicate the importance of the factors.  Further analysis could perform multiple regression to model the callback proportion as a function of several variables.  The importance of each variable could be determined by comparing their standardized regression coefficients.  The standardized coefficients compare the relative influence that perturbations in each variable have on callback proportions.  Although regression coefficients cannot necessarily be compared directly because they may have different units, standardized coefficients share the same scale and so they can be compared directly.  Variables with larger standardized coefficients are considered more important for callback rates.
</p>
</div>